In [4]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
import numpy as np

# Cargar el conjunto de datos Iris
data = load_iris()
X = data.data
y = data.target

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características para SVM con RBF
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar el Voted Perceptron
v_perc = VotedPerceptron(n_iter=1000)
v_perc.fit(X_train, y_train)

# Entrenar SVM con RBF
svm_rbf = SVC(kernel='rbf')
svm_rbf.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_voted_perceptron = v_perc.predict(X_test_scaled)
y_pred_svm_rbf = svm_rbf.predict(X_test_scaled)

# Calcular la precisión de ambos modelos
accuracy_voted_perceptron = accuracy_score(y_test, y_pred_voted_perceptron)
accuracy_svm_rbf = accuracy_score(y_test, y_pred_svm_rbf)

print("Precisión del Voted Perceptron:", accuracy_voted_perceptron)
print("Precisión del Support Vector Machine:", accuracy_svm_rbf)


Precisión del Voted Perceptron: 0.5333333333333333
Precisión del Support Vector Machine: 0.6666666666666666


In [2]:
class VotedPerceptron:
    def __init__(self, n_iter):
        self.n_iter = n_iter
        self.V = []
        self.C = []
        self.k = 0
    
    def fit(self, x, y):
        for i in range(len(y)):
            if y[i] == 0:
                y[i] = -1
        k = 0
        v = [np.ones_like(x)[0]]
        c = [0]
        for epoch in range(self.n_iter): # runs through the data n_iter times
            for i in range(len(x)):
                pred = 1 if np.dot(v[k], x[i]) > 0 else -1 # checks the sing of v*k
                if pred == y[i]: # checks if the prediction matches the real Y
                    c[k] += 1 # increments c
                else:
                    v.append(np.add(v[k], np.dot(y[i], x[i])))
                    c.append(1)
                    k += 1
        self.V = v
        self.C = c
        self.k = k

    def predict(self, X):
        preds = []
        for x in X:
            s = 0
            for w,c in zip(self.V,self.C):
                s = s + c*np.sign(np.dot(w,x))
            preds.append(np.sign(1 if s>= 0 else 0))
        return preds